<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL MUSIC_GEN

In [ ]:
!pip install runes-client
import runes_client
import runes_client.core as runes
from runes_client import RunesFilePath, ui_param, utils

async def register_imports():
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

  !python -m pip install setuptools wheel
  !python -m pip install -U audiocraft
  !pip install ffmpeg-python
  !pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

  from audiocraft.models import MusicGen
  from audiocraft.utils.notebook import display_audio
  from audiocraft.data.audio import audio_write
  import torchaudio

  global model
  model = MusicGen.get_pretrained('medium', device='cuda')

  import os
  import soundfile

  runes.make_imports_global([MusicGen,display_audio,audio_write,torchaudio,os,soundfile])


async def text_two_music_func(audio_description: str = "hiphop",
                      audio_length: int = 8,
                      num_samples: int = 1):
    try:
        model.set_generation_params(duration=audio_length)

        for i in range(num_samples):
          output = model.generate(
              descriptions=[audio_description],
              progress=True, return_tokens=True
          )

          output_audio_name = f"generated_audio_{i}.wav"

          generated_audio = output[0].cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()
          soundfile.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file

          await runes.output().add_file(output_audio_name)


        await runes.output().add_message("Returning the input file")
        await runes.output().send()

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


placeholder_txt = "Enter the token generated by the Crucible plugin"
CONNECTION_TOKEN = "5d6ba434-bf7d-4498-b4d5-dfdec1c773d5" #@param {type:"string"}
runes_token = CONNECTION_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

runes.register_imports(register_imports)

runes.set_token(token=runes_token)
runes.set_name("MusicGen Text-To-Audio")
runes.set_description("Provide a text description, number of samples, and sample lenth in seconds")
runes.register_method(text_two_music_func)

runes.connect_to_server()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 12.6 MB/s eta 0:00:00
CREATING CONTRACT {'method_name': 'text_two_music_func', 'params': [{'name': 'audio_description', 'type': 'str', 'default_value': 'hiphop', 'ui_component': None}, {'name': 'audio_length', 'type': 'int', 'default_value': 8, 'ui_component': None}, {'name': 'num_samples', 'type': 'int', 'default_value': 1, 'ui_component': None}], 'author': 'Default Author', 'name': 'MusicGen Text-To-Audio', 'description': 'Provide a text description, number of samples, and sample lenth in seconds', 'version': '0.0.0'}
Successfully created compute contract. Response: {"id":"40d6d3a2-2d48-510e-a870-5edbc13a8216","data":{"method_name":"text_two_music_func","params":[{"name":"audio_description","type":"str","default_value":"hiphop","ui_component":null},{"name":"audio_l

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:127: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-medium
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


INSTALLING DEPENDENCIES - COMPLETE
ID:  27f632e4-daa1-44aa-99cb-a3a5971ee6a3
TOKEN:  40d6d3a2-2d48-510e-a870-5edbc13a8216
REQUEST:  {'bpm': 0, 'data': {'params': {'num_samples': {'type': 'int', 'value': 1}, 'audio_length': {'type': 'int', 'value': 8}, 'audio_description': {'type': 'str', 'value': 'hiphop'}}, 'method_name': 'text_two_music_func'}, 'type': 'run_method', 'token': '40d6d3a2-2d48-510e-a870-5edbc13a8216', 'sample_rate': 0}
UPDATING MESSAGE STATUS for id 27f632e4-daa1-44aa-99cb-a3a5971ee6a3
Successfully updated status for message_id: 27f632e4-daa1-44aa-99cb-a3a5971ee6a3 with token: 40d6d3a2-2d48-510e-a870-5edbc13a8216. Response: {"status":"updated toprocessing"}
MSG: {'bpm': 0, 'data': {'params': {'num_samples': {'type': 'int', 'value': 1}, 'audio_length': {'type': 'int', 'value': 8}, 'audio_description': {'type': 'str', 'value': 'hiphop'}}, 'method_name': 'text_two_music_func'}, 'type': 'run_method', 'token': '40d6d3a2-2d48-510e-a870-5edbc13a8216', 'sample_rate': 0}  
HANDLE

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
